In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz

     |████████████████████████████████| 827.9 MB 4.5 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.0-py3-none-any.whl size=829187827 sha256=82abf6e9f00e9af17599384282523782aceb1cd4d2cf47dbcf9970db5e03a3ad
  Stored in directory: /root/.cache/pip/wheels/4b/6e/87/8b129988ba4d66c064415e09db36d5945b638aabcaf8f41765
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.3 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529.csv   a0605.csv	        heatmap531.ipynb
 a0530.csv   a0606.csv	        Heat_Map_Online_language.ipynb
 a0531.csv   csv	        __MACOSX
 a0601.csv   csv.zip	        non_violant1.csv
 a0602.csv   dataframe_csv      non_violant2.csv
 a0603.csv   Github	        violant.csv
 a0604.csv   heatmap530.ipynb  '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_607 = 'csv/US06_07.csv'

In [8]:
ct = pd.read_csv(ct_path_607)

In [9]:
ct.shape[1]

47

In [10]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,249695635088304_3331265190264651,Facebook,2020-06-07 19:46:29,2021-06-17 18:54:01,photo,Why It's Impossible to Get Rid of Bad Cops,thestreet.com,"Derek Chauvin, the officer who mudered George ...",PRAYER TO OVERCOME RACISM: In his message to o...,7.0,https://www.facebook.com/stdominicer/photos/a....,https://www.facebook.com/249695635088304/posts...,1772,2.500000,1.0,34,9,5,17,0,0,0,0,0,1,10,2,4,6,2,0,2,0,0,2,11515193,"St. Dominic Church, Eagle Rock",stdominicer,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"The Catholic parish of Eagle Rock, a neighborh...",False,2011-11-20 16:26:16,en,0,11515193|3331265190264651
1,803355826381154_3223243767725669,Facebook,2020-06-07 19:46:28,2021-08-29 22:27:44,link,GOLDSTEIN: Media deliberately distorted what T...,torontosun.com,Contrary to a globally reported blunder by the...,Here’s what Trump said: “Equal justice under t...,2.0,https://torontosun.com/opinion/columnists/gold...,https://www.facebook.com/803355826381154/posts...,8203,2.315789,1.0,20,12,3,0,0,0,3,6,0,0,2,2,2,2,1,2,2,2,0,1,8504563,WINN: World Integrity News Network,WINNNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,An entity of World Integrity News Network (WI...,False,2014-08-29 07:32:42,en,0,8504563|3223243767725669
2,49680588265_10159922056138266,Facebook,2020-06-07 19:46:12,2021-08-26 10:28:23,link,Protesters gather for Black Lives Matter rally...,stevenspointjournal.com,Sunday's protest is the third to occur in Stev...,This is the third Black Lives Matter protest t...,1.0,https://www.stevenspointjournal.com/story/news...,https://www.facebook.com/49680588265/posts/101...,11500,5.269231,1.0,67,16,29,21,0,1,1,2,0,0,5,3,4,3,2,4,2,3,0,2,13672,Stevens Point Journal Media,StevensPointJRL,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Stevens Point Journal Media is the only daily ...,True,2008-12-01 23:39:04,en,0,13672|10159922056138266
3,554234847924408_567752680831724,Facebook,2020-06-07 19:46:08,2021-07-14 21:28:53,live_video_complete,Protesters gather for Black Lives Matter rally...,stevenspointjournal.com,Sunday's protest is the third to occur in Stev...,Live at the George Floyd Memorial Location,1.0,https://www.facebook.com/KidDr1ft/videos/56775...,https://www.facebook.com/554234847924408/posts...,13460,-3.053763,2.0,31,8,35,19,0,0,0,0,0,0,21,21,223,10,2,3,2,2,0,2,1306557,KidDr1ft,KidDr1ft,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"Music Producer, Recording Hip-Hop artist, Adve...",True,2012-11-19 21:12:15,en,0,1306557|567752680831724
4,1151043328247924_3288525607833008,Facebook,2020-06-07 19:46:07,2021-08-16 11:11:19,link,Minnesota Officials Link Arrested Looters to '...,courthousenews.com,As Minneapolis underwent a fourth night of pro...,White Supremacy Bloggers and media need to be ...,1.0,https://www.courthousenews.com/minnesota-offic...,https://www.facebook.com/1151043328247924/post...,7216,1.836735,1.0,30,43,5,2,1,0,3,6,0,0,10,6,2,11,2,1,15,2,0,4,8517732,Representative Ruth Buffalo,RuthBuffalo4ND,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,District 27- House Representative,True,2016-04-15 22:02:13,en,0,8517732|3288525607833008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,"{'Houston': 1, 'Mass.': 1}",{}
1,{},{'U.S.': 1}
2,{},{}
3,{},{}
4,{},"{'Minneapolis': 1, 'Minnesota': 1}"


In [24]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,249695635088304_3331265190264651,Facebook,2020-06-07 19:46:29,2021-06-17 18:54:01,photo,Why It's Impossible to Get Rid of Bad Cops,thestreet.com,"Derek Chauvin, the officer who mudered George ...",PRAYER TO OVERCOME RACISM: In his message to o...,7.0,https://www.facebook.com/stdominicer/photos/a....,https://www.facebook.com/249695635088304/posts...,1772,2.500000,1.0,34,9,5,17,0,0,0,0,0,1,10,2,4,6,2,0,2,0,0,2,11515193,"St. Dominic Church, Eagle Rock",stdominicer,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"The Catholic parish of Eagle Rock, a neighborh...",False,2011-11-20 16:26:16,en,0,11515193|3331265190264651,"{'Houston': 1, 'Mass.': 1}",{},{}
1,803355826381154_3223243767725669,Facebook,2020-06-07 19:46:28,2021-08-29 22:27:44,link,GOLDSTEIN: Media deliberately distorted what T...,torontosun.com,Contrary to a globally reported blunder by the...,Here’s what Trump said: “Equal justice under t...,2.0,https://torontosun.com/opinion/columnists/gold...,https://www.facebook.com/803355826381154/posts...,8203,2.315789,1.0,20,12,3,0,0,0,3,6,0,0,2,2,2,2,1,2,2,2,0,1,8504563,WINN: World Integrity News Network,WINNNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,An entity of World Integrity News Network (WI...,False,2014-08-29 07:32:42,en,0,8504563|3223243767725669,{},{'U.S.': 1},{}
2,49680588265_10159922056138266,Facebook,2020-06-07 19:46:12,2021-08-26 10:28:23,link,Protesters gather for Black Lives Matter rally...,stevenspointjournal.com,Sunday's protest is the third to occur in Stev...,This is the third Black Lives Matter protest t...,1.0,https://www.stevenspointjournal.com/story/news...,https://www.facebook.com/49680588265/posts/101...,11500,5.269231,1.0,67,16,29,21,0,1,1,2,0,0,5,3,4,3,2,4,2,3,0,2,13672,Stevens Point Journal Media,StevensPointJRL,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Stevens Point Journal Media is the only daily ...,True,2008-12-01 23:39:04,en,0,13672|10159922056138266,{},{},{}
3,554234847924408_567752680831724,Facebook,2020-06-07 19:46:08,2021-07-14 21:28:53,live_video_complete,Protesters gather for Black Lives Matter rally...,stevenspointjournal.com,Sunday's protest is the third to occur in Stev...,Live at the George Floyd Memorial Location,1.0,https://www.facebook.com/KidDr1ft/videos/56775...,https://www.facebook.com/554234847924408/posts...,13460,-3.053763,2.0,31,8,35,19,0,0,0,0,0,0,21,21,223,10,2,3,2,2,0,2,1306557,KidDr1ft,KidDr1ft,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"Music Producer, Recording Hip-Hop artist, Adve...",True,2012-11-19 21:12:15,en,0,1306557|567752680831724,{},{},{}
4,1151043328247924_3288525607833008,Facebook,2020-06-07 19:46:07,2021-08-16 11:11:19,link,Minnesota Officials Link Arrested Looters to '...,courthousenews.com,As Minneapolis underwent a fourth night of pro...,White Supremacy Bloggers and media need to be ...,1.0,https://www.courthousenews.com/minnesota-offic...,https://www.facebook.com/1151043328247924/post...,7216,1.836735,1.0,30,43,5,2,1,0,3,6,0,0,10,6,2,11,2,1,15,2,0,4,8517732,Representative Ruth Buffalo,RuthBuffalo4ND,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,District 27- House Representative,True,2016-04-15 22:02:13,en,0,8517732|32885

In [25]:
ct.to_csv('a0607.csv')

In [26]:
ct=pd.read_csv('a0607.csv',lineterminator='\n')

In [27]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [28]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [29]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,"{'Houston': 1, 'Mass.': 1}",{},{}
1,{},{'U.S.': 1},{}
4,{},"{'Minneapolis': 1, 'Minnesota': 1}",{'Minnesota': 1}
5,{},"{'Minneapolis': 1, 'Minnesota': 1}",{'Minnesota': 1}
6,{},{'U.S.': 1},{}


In [30]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [31]:
ct_ner['NER-msg']

0       [Mass., Houston]
1                     []
4                     []
5                     []
6                     []
              ...       
8976                  []
8977           [Florida]
8978                  []
8979                  []
8981                  []
Name: NER-msg, Length: 6349, dtype: object

In [32]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
ct_ner['NER']

0                                        [Mass., Houston]
1                                                  [U.S.]
4                     [Minneapolis, Minnesota, Minnesota]
5                     [Minneapolis, Minnesota, Minnesota]
6                                                  [U.S.]
                              ...                        
8976             [Minneapolis, Miramar, Florida, Florida]
8977                      [Florida, Minneapolis, Florida]
8978                           [US, Minnesota, Edinburgh]
8979                           [US, Minnesota, Edinburgh]
8981    [Manistee\n, Beulah, Charlotte, Corunna, Wyomi...
Name: NER, Length: 6349, dtype: object

In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
ner_lst = ct_ner['NER'].values

In [37]:
ner_lst

array([list(['mass.', 'houston']), list(['u.s.']),
       list(['minneapolis', 'minnesota']), ...,
       list(['us', 'minnesota', 'edinburgh']),
       list(['us', 'minnesota', 'edinburgh']),
       list(['manistee\n', 'beulah', 'charlotte', 'corunna', 'wyoming', 'greenville', 'saginaw', 'grand rapids', 'holland', 'owosso'])],
      dtype=object)

In [38]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [39]:
list

['mass.',
 'houston',
 'u.s.',
 'minneapolis',
 'minnesota',
 'minneapolis',
 'minnesota',
 'u.s.',
 'upper new york',
 'massachusetts',
 'massachusetts',
 'el paso',
 'minneapolis',
 'minnesota',
 'minneapolis',
 'minnesota',
 'minneapolis',
 'minnesota',
 'the united states',
 'america',
 'us',
 'charlotte',
 'north carolina',
 'broward county',
 'minneapolis',
 'miami',
 'carol city',
 'florida state',
 'staten island',
 'america',
 'america',
 'minneapolis',
 'portland city',
 'nc state',
 'charlotte',
 'panthers',
 'n.c.',
 'brunswick',
 'georgia',
 'florida',
 'myrtle beach',
 'honolulu',
 'portland city',
 'ahmaud arbery',
 'houston',
 'ahmaud arbery',
 'washington',
 'the united states',
 'd.c.',
 'd-conn.',
 'the united states',
 'the united states',
 'lakeland',
 'the united states',
 'alabama',
 'minneapolis',
 'louisville',
 'lancaster',
 'usa',
 'the united states of america',
 'usa',
 'the united states of america',
 'athens',
 'encinitas',
 'san diego',
 'savannah',
 'ne

In [40]:
c = collections.Counter()

In [41]:
dict_a=collections.Counter(list).most_common()

In [42]:
dict_a

[('minneapolis', 1821),
 ('america', 583),
 ('washington', 448),
 ('u.s.', 356),
 ('houston', 344),
 ('the united states', 323),
 ('d.c.', 281),
 ('us', 276),
 ('minnesota', 269),
 ('texas', 220),
 ('new york city', 218),
 ('new york', 200),
 ('north carolina', 184),
 ('santa cruz', 165),
 ('los angeles', 158),
 ('chicago', 156),
 ('california', 140),
 ('philadelphia', 125),
 ('dc', 122),
 ('atlanta', 116),
 ('london', 110),
 ('seattle', 98),
 ('nyc', 96),
 ('buffalo', 91),
 ('connecticut', 91),
 ('san francisco', 85),
 ('canada', 78),
 ('denver', 75),
 ('portland', 69),
 ('raeford', 68),
 ('milwaukee', 67),
 ('massachusetts', 66),
 ('maryland', 63),
 ('boston', 56),
 ('san diego', 53),
 ('alabama', 52),
 ('illinois', 52),
 ('louisville', 51),
 ('ny', 50),
 ('birmingham', 46),
 ('uk', 45),
 ('austin', 45),
 ('sacramento', 44),
 ('oakland', 43),
 ('pittsburgh', 42),
 ('usa', 39),
 ('n.c.', 38),
 ('florida', 38),
 ('iowa', 35),
 ('colorado', 35),
 ('san antonio spurs', 34),
 ('virginia',

In [43]:
dataframe607=pd.DataFrame(dict_a)

In [44]:
len(dict_a)

1580

In [45]:
dataframe607.to_csv('dataframe0607.csv')